In [ ]:
# ws08
from sklearn import datasets 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

digits = datasets.load_digits()

In [ ]:
n_samples = len(digits.images) 
X = digits.images.reshape((n_samples, -1))

X[0]

In [ ]:
X.shape

In [ ]:
y = digits.target
y[:15]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA()
X_pca = pca.fit_transform(X)

In [ ]:
pca.n_components_

In [ ]:
pca

In [ ]:
X_pca.shape

In [ ]:
print(pca.components_.shape)

In [ ]:
exp_variance = pca.explained_variance_ratio_
exp_variance

In [ ]:
sns.set()

x = list(range(1, len(exp_variance)+1))
sns.lineplot(x, exp_variance, marker='s', markersize=4)
plt.title('Explained Varience')
plt.xlabel('Number of components')

plt.show()

In [ ]:
import numpy as np

x = list(range(1, len(exp_variance)+1))
sns.lineplot(x, np.cumsum(pca.explained_variance_ratio_)) 

plt.xlabel('Number of components')
plt.ylabel('Cumulative Explained Variance');
plt.title('PCA Explained Variance Ratio')

plt.show()

In [ ]:
pca_num = PCA(.95)
pca_num.fit(X)
pca_num.n_components_

In [ ]:
n_comp = 2
pca = PCA(n_components=n_comp)
pca.fit(X)
print('Explained variance by %i components: %0.3f' % 
      (n_comp, np.sum(pca.explained_variance_ratio_)))

In [ ]:

X_pca = pca.transform(X)
X_pca.shape

In [ ]:
df = pd.DataFrame(X_pca, columns=['pca1','pca2'])
df.head()

In [ ]:
df['labels'] = y
df.head()

In [ ]:
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = 5, 5  
g=sns.scatterplot('pca1', 'pca2', data=df, hue='labels', 
                  s=50, palette='Set1', legend='full')
g.legend(loc='center right', bbox_to_anchor=(1.4, 0.5), ncol=1)
plt.title('PCA: 2 Components (1,2)')
plt.show()

In [ ]:
df.species.unique()

In [ ]:
# split
from sklearn.model_selection import train_test_split
X_train, X_test,y_train,y_test = train_test_split(X, y, test_size=0.25, random_state=0) # 
X_train.shape , X_test.shape

In [ ]:

from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier

n_comp = 25
pipe = Pipeline([

                 ('pca', PCA(n_components=n_comp, whiten=True)),
                ('svc',SVC(gamma='scale'))
               ])

pipe.fit(X_train, y_train)

cvs = cross_val_score(pipe, X, y, cv=10)  
print('cross val scores {}'.format(cvs.round(2)))
print('Average (%) = {:.2f}' .format(cvs.mean() * 100 ))

In [ ]:
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

pipe.fit(X_train, y_train)

y_pred = pipe.predict(X_test)

print('Accuracy Score: {:.4f}'.format(accuracy_score(y_test, y_pred)))

print(classification_report(y_test, y_pred)) 

cm = confusion_matrix(y_test, y_pred)
print(cm)

In [ ]:
import scikitplot as skplot

y_pred = pipe.predict(X_test)
skplot.metrics.plot_confusion_matrix(y_test, y_pred, hide_zeros=True)

plt.show()

In [ ]:
pipe.predict(X_test)
pipe.predict([X[5]])
pipe.predict([X[15]])
pipe.predict([X[25]])

In [ ]:

import numpy as np
from PIL import Image, ImageOps

def predict_digit(imgfile): 
    img = Image.open('/folder/'+imgfile).convert('L') 
    img = ImageOps.invert(img)
    img = img.resize((8, 8), Image.ANTIALIAS) 

    pixel = np.array(img)
    pixel = pixel / 255.0 * 16
    pixel = pixel.astype('int')
    
    my_digit = pixel.reshape(1, -1)
    predicted = pipe.predict(my_digit) 
   
    plt.figure(figsize=(1,1))
    plt.imshow(pixel, cmap=plt.cm.gray_r)
    plt.title('Predicted:{}'. format(predicted[0]))
    plt.xticks([])
    plt.yticks([])    
    plt.show()

imgfiles = ['digit1.png','digit2.png','digit3.png','digit4.png','digit5.png',
            'digit6.png','digit7.png','digit8.png','digit9.png','digit0.png']
for i in imgfiles:
    predict_digit(i)